In [1]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import obb_login
from openbb import obb
from humbldata.core.utils.env import Env
import polars as pl

# obb_login()
obb.account.login(pat=Env().OBB_PAT, remember_me=True)

In [ ]:

from humbldata.toolbox.technical.mandelbrot_channel.model import (
    calc_mandelbrot_channel,
    calc_mandelbrot_channel_historical,
)


data = (
    obb.equity.price.historical(
        ["AAPL"],
        provider="yfinance",
        start_date="2020-01-01",
        end_date="2024-01-01",
        adjustment="splits_and_dividends",
    ).to_polars()
).drop(["dividends", "stock_splits"]).with_columns(pl.lit("AAPL").alias("symbol"))

mandelbrot = calc_mandelbrot_channel_historical(  # noqa: ERA001
        data,
        window="1m",
        rv_adjustment=True,
        rv_method="std",
        rv_grouped_mean=False,
        rs_method="RS",
        live_price=False,
    ).collect()
mandelbrot

# Setting up UserTable

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.standard_models.portfolio.analytics.user_table import UserTableQueryParams
from humbldata.core.utils.openbb_helpers import get_latest_price


symbols = UserTableQueryParams(symbol="AAPL, MSFT, NVDA").symbol
symbols



get_latest_price(symbols)

# Tesing Async


## Get Sector Async

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import aget_etf_sector, aget_equity_sector
import polars as pl

stocks = pl.Series([
    "GOOGL",  # Communication Services
    "AMZN",   # Consumer Discretionary
    "KO",     # Consumer Staples
    "XOM",    # Energy
    "JPM",    # Financials
    "JNJ",    # Health Care
    "HON",    # Industrials
    "AAPL",   # Information Technology
    "LIN",    # Materials
    "PLD",    # Real Estate
    "NEE"     # Utilities
])
# test = (await aget_etf_sector(symbols=stocks, provider="yfinance")).collect()
test = (await aget_etf_sector(symbols=["XLE", "AAPL"], provider="yfinance")).collect()
# test2 = (await aget_equity_sector(symbols=stocks, provider="yfinance")).collect()
# test2 = (await aget_equity_sector(symbols=["XLE", "AAPL"], provider="yfinance")).collect()
test

In [ ]:
symbols = ["CORN", 'SLV', 'DBA', "XLE", "AAPL", "FXE", "SPY", "QQQ", "BITO", "MAXI", "SATO", "BIL", "LQD", "QAI", "MNA", "CLSE"]
symbols2 = ["AAPL", "FXE", "SPY", "MAXI", "GLD"]
symbols3 = ['IBIT', 'GBTC', 'FBTC', 'ARKB', 'BITB', 'HODL', 'BRRR', 'UUP', 'BTCO', 'EZBC', 'FXY', 'USDU', 'FXE', 'FXF', 'BTCW', 'FXA', 'FXC', 'UDN', 'FXB', 'BTF', 'YBIT', 'YBTC', 'EFUT', 'MAXI', 'ARKA', 'AETH', 'ARKZ', 'BITC', 'DEFI', 'BTOP', 'CEW', 'ARKY', 'ARKC', 'BTRN', 'SETH', 'BETE', 'BETH', 'EETH', 'BITI']

df = obb.etf.info(symbol=symbols2, provider='yfinance').to_polars()
df

In [ ]:
from humbldata.core.utils.openbb_helpers import normalize_asset_class


normalized_df = normalize_asset_class(df)
normalized_df

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import aget_asset_class


(await aget_asset_class(symbols=["AAPL", "XLE"])).collect()

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.analytics.user_table.helpers import aget_asset_class_filter


(await aget_asset_class_filter(symbols=["AAPL", "XLE"])).collect()

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.constants import ASSET_CLASS_MAPPING


remapped_assets = df.with_columns(
            pl.col("category").replace(ASSET_CLASS_MAPPING)
        )
remapped_assets

In [ ]:
import polars as pl
import re

# Define a subset of ASSET_CLASS_MAPPING with only regex patterns
REGEX_ASSET_CLASS_MAPPING = {
    r".*\s?Bond.*": "Fixed Income",
    r".*Commodities.*": "Commodity",
    r".*Digital.*": "Crypto"
}

# Create a test DataFrame
df = pl.DataFrame({
    "category": [
        "Corporate Bond",
        "Government Bond",
        "Energy Commodities",
        "Agricultural Commodities",
        "Regular Stock",
        "Bond Fund",
        "Digital Currency"
    ]
})

# Function to apply regex mapping
def apply_regex_mapping(category):
    for pattern, asset_class in REGEX_ASSET_CLASS_MAPPING.items():
        if re.match(pattern, category):
            return asset_class
    return category

# Apply the regex mapping
# %timeit df.with_columns([pl.col("category").map_elements(apply_regex_mapping).alias("mapped_category")])
%timeit df.with_columns(pl.col("category").replace(REGEX_ASSET_CLASS_MAPPING))
result = df.with_columns(pl.col("category").str.replace(r".*\s?Bond.*", "Fixed Income"))


print(result)

## Test aget_sector_filter

In [4]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.analytics.user_table.helpers import aget_sector_filter


(await aget_sector_filter(symbols=["XLU", "XLE", "XLF", "XLC", "KO"])).collect()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


symbol,sector
str,str
"""KO""","""Consumer Defensive"""
"""XLU""","""Utilities"""
"""XLE""","""Energy"""
"""XLF""","""Financials"""
"""XLC""","""Communication Services"""


# Test aggregate_user_table_data

In [ ]:
from humbldata.portfolio.analytics.user_table.helpers import aggregate_user_table_data


(await aggregate_user_table_data(symbols=["XLU", "XLE", "AAPL"])).collect()

In [ ]:
from humbldata.core.utils.openbb_helpers import aget_latest_price


(await aget_latest_price(symbol=["AAPL", "NVDA", "XLE"])).collect()